<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Luis Figueroa
- Nombre de alumno 2: Felipe Méndez


### **Link de repositorio de GitHub:** `https://github.com/Felipe-mendezp/Labs-Figueroa-Mendez`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [40]:
# Librería Core del lab.
import os
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import RandomOverSampler

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go



# Librería para NLP
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [41]:
# # Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = '/content/drive/MyDrive/Laboratorios'
# except: 
#     print('Ignorando conexión drive-colab')

os.chdir(r'C:\Users\luish\OneDrive - Universidad de Chile\Universidad de Chile - FCFM\XI Semestre\MDS7202 - Laboratorio de Programacion Cientifica para Ciencia de Datos\Labs\Lab_09')

In [42]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [43]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [44]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [45]:
# Definimos algunas stopword que queremos que sean eliminadas
stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [46]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [47]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [48]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [X] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [X] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [49]:
# Atributos de interés
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

# Tokenizador
tokenizador = StemmerTokenizer()

# Bag of Words
bog_transformer = CountVectorizer(tokenizer=tokenizador, ngram_range=(1, 2))

# MinMaxScaler
scaler_transformer = MinMaxScaler()

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('bag_of_words', bog_transformer, 'history_text'),
        ('scaling', scaler_transformer, atributos_de_interes)
    ])



# Aplicamos el pipeline
features = preprocessor.fit_transform(df_comics)

c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [X] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [X] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [X] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [50]:


# Separar el conjunto de datos en train y test
X = df_comics.drop('alignment', axis=1)  # Características
y = df_comics['alignment']  # Etiquetas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True,stratify=y, random_state=1234)




In [51]:
# Creamos el pipeline
pipeline_nb = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectPercentile(score_func=f_classif, percentile=90)),
    ('classifier', MultinomialNB())
])


# Entrenamos el modelo
pipeline_nb.fit(X_train, y_train)

# Predicción de las etiquetas en test
y_pred = pipeline_nb.predict(X_test)

# Classification_report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

         Bad       0.64      0.08      0.14        86
        Good       0.59      0.97      0.73       148
     Neutral       1.00      0.04      0.08        23

    accuracy                           0.59       257
   macro avg       0.74      0.37      0.32       257
weighted avg       0.64      0.59      0.48       257



In [52]:
# Creamos un DummyClassifier estratificado
dummy = DummyClassifier(strategy='stratified')

pipeline_dummy = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectPercentile(score_func=f_classif, percentile=90)),
    ('classifier', dummy)
])


# Entrenamos el DummyClassifier
pipeline_dummy.fit(X_train, y_train)

# Predicción de las etiquetas para el conjunto de test
y_pred_dummy = pipeline_dummy.predict(X_test)

# Classification_report
report_dummy = classification_report(y_test, y_pred_dummy)
print("Classification Report (DummyClassifier):")
print(report_dummy)

c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classification Report (DummyClassifier):
              precision    recall  f1-score   support

         Bad       0.42      0.41      0.41        86
        Good       0.58      0.59      0.59       148
     Neutral       0.04      0.04      0.04        23

    accuracy                           0.48       257
   macro avg       0.35      0.35      0.35       257
weighted avg       0.48      0.48      0.48       257




En la comparativa entre el modelo de clasificación de texto Naive Bayes y el modelo dummy, se evidenció claramente que el modelo Naive Bayes obtuvo resultados superiores en términos de accuracy y precisión. Esta superioridad se debe a varias razones fundamentales. En primer lugar, Naive Bayes es un algoritmo que aprovecha la teoría estadística del teorema de Bayes para calcular la probabilidad condicional de cada clase dado un conjunto de características. Esto le permite capturar de manera más efectiva las relaciones subyacentes entre las palabras y las etiquetas de clasificación. Por otro lado, el modelo dummy es una técnica de referencia simple que asigna las clases de manera aleatoria o siguiendo una distribución uniforme. Como resultado, carece de la capacidad de aprender patrones o extraer información relevante del conjunto de datos. 

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [66]:


classifiers = [
    {
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.1, 0.5, 1.0]
    },
    {
        'classifier': [RandomForestClassifier(random_state=1234)],
        'classifier__n_estimators': [100, 200, 500],
        'classifier__max_depth': [None, 100, 500]
    },
    {
        'classifier': [LogisticRegression(random_state=1234)],
        'classifier__C': [0.5, 1,  5],
        'classifier__solver': ['liblinear', 'saga']
    }
]

count_vectorizer_params = {
    'preprocessor__bag_of_words__ngram_range': [(1, 1), (1, 2), (1, 3)]
}

percentiles = [20, 40, 60, 80]

param_grid = [
    {
        **classifier,
        **count_vectorizer_params,
        'selector__percentile': percentiles
    }
    for classifier in classifiers
]

# Crear el pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectPercentile(score_func=f_classif, percentile=90)),
    ('classifier', dummy)
])


In [67]:
# Crear el HalvingGridSearchCV
halving_search = HalvingGridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    n_jobs=-1,
    cv=5,
    factor=2,
    random_state=1234,
    verbose = 10
)

# Ejecutar la búsqueda en la grilla
halving_search.fit(X_train, y_train)

n_iterations: 6
n_required_iterations: 8
n_possible_iterations: 6
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 216
n_resources: 30
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
----------
iter: 1
n_candidates: 108
n_resources: 60
Fitting 5 folds for each of 108 candidates, totalling 540 fits
----------
iter: 2
n_candidates: 54
n_resources: 120
Fitting 5 folds for each of 54 candidates, totalling 270 fits
----------
iter: 3
n_candidates: 27
n_resources: 240
Fitting 5 folds for each of 27 candidates, totalling 135 fits
----------
iter: 4
n_candidates: 14
n_resources: 480
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 5
n_candidates: 7
n_resources: 960
Fitting 5 folds for each of 7 candidates, totalling 35 fits


c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\luish\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('bag_of_words',
                                                                                CountVectorizer(ngram_range=(1,
                                                                                                             2),
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x00000132BB5A9450>),
                                                                                'history_text'),
                                                                               ('scaling',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score...
                                                                              1),
                                                                             (1,
                                                                              2),
                                                                             (1,
                                                                              3)],
                                 'selector__percentile': [20, 40, 60, 80]},
                                {'classifier': [LogisticRegression(C=0.5,
                                                                   random_state=1234,
                                                                   solver='liblinear')],
                                 'classifier__C': [0.5, 1, 5],
                                 'classifier__solver': ['liblinear', 'saga'],
                                 'preprocessor__bag_of_words__ngram_range': [(1,
                                                                              1),
                                                                             (1,
                                                                              2),
                                                                             (1,
                                                                              3)],
                                 'selector__percentile': [20, 40, 60, 80]}],
                    random_state=1234, scoring='accuracy', verbose=10)

In [68]:

# Obtener los mejores parámetros y el mejor estimador
best_params = halving_search.best_params_
print(best_params)
best_estimator = halving_search.best_estimator_
print(best_estimator)

{'classifier': LogisticRegression(C=0.5, random_state=1234, solver='liblinear'), 'classifier__C': 0.5, 'classifier__solver': 'liblinear', 'preprocessor__bag_of_words__ngram_range': (1, 1), 'selector__percentile': 60}
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('bag_of_words',
                                                  CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x00000132CDD87250>),
                                                  'history_text'),
                                                 ('scaling', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_s

In [69]:
# Obtener las predicciones en el conjunto de entrenamiento y prueba
y_train_pred = best_estimator.predict(X_train)
y_test_pred = best_estimator.predict(X_test)

# Evaluar el rendimiento en el conjunto de entrenamiento
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Evaluar el rendimiento en el conjunto de prueba
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

Train Classification Report:
              precision    recall  f1-score   support

         Bad       0.99      0.99      0.99       343
        Good       0.99      1.00      0.99       595
     Neutral       1.00      0.97      0.98        90

    accuracy                           0.99      1028
   macro avg       0.99      0.98      0.99      1028
weighted avg       0.99      0.99      0.99      1028

Test Classification Report:
              precision    recall  f1-score   support

         Bad       0.67      0.55      0.60        86
        Good       0.73      0.90      0.81       148
     Neutral       0.33      0.09      0.14        23

    accuracy                           0.71       257
   macro avg       0.58      0.51      0.52       257
weighted avg       0.68      0.71      0.68       257



In [75]:
# Obtener los parámetros utilizados en el CountVectorizer
count_vectorizer_params = best_estimator.named_steps['preprocessor'].named_transformers_['bag_of_words'].get_params()['ngram_range']
# Obtener las columnas seleccionadas por el SelectPercentile
selected_columns = best_estimator.named_steps['selector'].get_support(indices=True)
# Obtener el percentil utilizado por SelectPercentile
percentile = best_estimator.named_steps['selector'].percentile

print('Best classifier:')
print(best_params['classifier'])

print("CountVectorizer parameters:")
print(count_vectorizer_params)
print("SelectPercentile percentile:", percentile)
print("Selected columns:")
print(len(selected_columns))

Best classifier:
LogisticRegression(C=0.5, random_state=1234, solver='liblinear')
CountVectorizer parameters:
(1, 1)
SelectPercentile percentile: 60
Selected columns:
12312


La mejor combinación encontrada en la HalvingGridSearchCV consiste en utilizar una Regresión Logística con un solver lineal y un valor de C igual a 0.5. El valor de C indica una regularización moderada. Además, se seleccionó un rango de n-gramas de (1,1) en el CountVectorizer, lo que significa que solo se consideran las palabras individuales. Esto permite capturar la importancia de las palabras individuales en la clasificación. Se seleccionaron el top 60% de variables a través de SelectPercentile, lo que implica que se mantienen las características más relevantes para la clasificación. En total, se retuvieron 12.312 columnas, lo que proporciona un conjunto de características considerablemente informativas. Estos parámetros se eligieron en función del rendimiento y la capacidad de generalización del modelo en el conjunto de datos utilizado en la HalvingGridSearchCV.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [71]:
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) 
df_comics_no_label.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,Prediction
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,Good
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Good
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,Good
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good


In [72]:
df_comics_no_label['Prediction'] = best_estimator.predict(df_comics_no_label)
df_comics_no_label[['name','real_name','history_text','Prediction']].sample(10)

,name,real_name,history_text,Prediction
47,Hulk 2099,John Eisenhart,"In one possible future, the Marvel Universe of...",Good
31,Destroyer (MCU),NaN,The Destroyer (or The Destroyer Automaton) was...,Bad
55,Living Totem,NaN,A stranded alien who crashed in the American s...,Good
66,Ronan The Accuser (MCU),Ronan,"Ronan, otherwise known as Ronan the Accuser, w...",Good
83,Batcow,None,Bat-Cow was originally a cow that was found by...,Good
39,Ghost Rider (SONY),Johnny Blaze,Johnny Blaze was a stunt motorcyclist who sold...,Good
26,Chosen Undead,NaN,After becoming an undead the player character ...,Good
60,Proxima Midnight,NaN,One of Thanos' generals in the Black Order,Bad
15,Banshee (FOX),Sean Cassidy,Banshee is first seen hitting on a girl in an ...,Bad
23,Caliban (FOX),NaN,"In 1983, Caliban worked as a dealer in the bla...",Good


In [73]:
df_comics_no_label[['name','real_name','history_text','Prediction']][df_comics_no_label.name.isin(['Batcow','Vergil','Gorilla Girl'])]

,name,real_name,history_text,Prediction
16,Batcow,None,Bat-Cow was originally a cow that was found by...,Good
40,Gorilla Girl,Fahnbullah Eddy,A carnival performer with the ability to turn ...,Good
78,Vergil,Vergil Sparda,"Vergil, later also known as Nelo Angelo, is on...",Good
83,Batcow,None,Bat-Cow was originally a cow that was found by...,Good


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>